In [1]:
import pandas as pd
import numpy  as np

   
import pickle
   

from scipy.sparse import issparse

 
from pathlib import Path


pd.set_option('display.max_columns', 5)

In [2]:
#### user specified
data_folder = "D:/analyze_Pearson_residuals/"

data_subfolder = "33k_PBMC"

data_path = Path ( data_folder + data_subfolder )

In [3]:
# output data
results_pkl = "Lg_complementary_samples.pkl"

# input data
counts_pkl = "counts_sparse.pkl" # sparse pandas data frame: cells in columns; genes in rows
gene_array_pkl  = "gene_array_complementary_samples.pkl"
df_cells_pkl  = "df_cells_complementary_samples.pkl" 


# path: output data
results_dsn = data_path / results_pkl

# paths: input data
counts_dsn = data_path / counts_pkl  
df_cells_dsn = data_path / df_cells_pkl
gene_array_dsn = data_path / gene_array_pkl

In [4]:
# https://github.com/scverse/scanpy/tree/master/scanpy/experimental/pp
# from   _normalization.py  rows 26-64

def _pearson_residuals(X, theta, clip, copy=False):    # removed check_values from parm list

    X = X.copy() if copy else X

    # check theta
    if theta <= 0:
        # TODO: would "underdispersion" with negative theta make sense?
        # then only theta=0 were undefined..
        raise ValueError('Pearson residuals require theta > 0')
    # prepare clipping
    if clip is None:
        n = X.shape[0]
        clip = np.sqrt(n)
    if clip < 0:
        raise ValueError("Pearson residuals require `clip>=0` or `clip=None`.")

# vfk 2022 08 16		
    # if check_values and not check_nonnegative_integers(X):
        # warn(
            # "`normalize_pearson_residuals()` expects raw count data, but non-integers were found.",
            # UserWarning,
        # )

    if issparse(X):
        sums_genes = np.sum(X, axis=0)
        sums_cells = np.sum(X, axis=1)
        sum_total = np.sum(sums_genes).squeeze()
    else:
        sums_genes = np.sum(X, axis=0, keepdims=True)
        sums_cells = np.sum(X, axis=1, keepdims=True)
        sum_total = np.sum(sums_genes)

    mu = np.array(sums_cells @ sums_genes / sum_total)
    diff = np.array(X - mu)
    residuals = diff / np.sqrt(mu + mu**2 / theta)

    # clip
    residuals = np.clip(residuals, a_min=-clip, a_max=clip)

    return residuals	
	
	
pctl_list = [.01,.05, .10, .25, .5, .75, .90, .95, .96, .97, .98, .99 ]

In [5]:
df_selected_cells = pd.read_pickle( df_cells_dsn )
print ( ' df_selected_cells:  ' )
print ( '\n', df_selected_cells )
print ( '\n\n df_selected_cells.sum:  ' )
print ( df_selected_cells.sum() )

f = open( gene_array_dsn, 'rb' )    
gene_select_array  = pickle.load(f)     
f.close()       
print ( '\n\n gene_select_array:  ' )
print ( gene_select_array )

 df_selected_cells:  

                       0      1
AAACATTGACGACT-1  False   True
AAACATTGACGGGA-1   True  False
AAACATTGCCGCTT-1   True  False
AAACATTGCTCATT-1   True  False
AAACCGTGCCCTCA-1  False   True
...                 ...    ...
TTTGCATGCACTAG-6   True  False
TTTGCATGCCGAAT-6  False   True
TTTGCATGGAGGTG-6  False   True
TTTGCATGGATAAG-6  False   True
TTTGCATGGGCATT-6   True  False

[33148 rows x 2 columns]


 df_selected_cells.sum:  
0    16593
1    16555
dtype: int64


 gene_select_array:  
['ENSG00000254709' 'ENSG00000163736' 'ENSG00000087086' ...
 'ENSG00000135549' 'ENSG00000155962' 'ENSG00000122025']


In [8]:
df_counts_sparse = pd.read_pickle ( counts_dsn )
print ( ' df_counts_sparse:' )
print ( df_counts_sparse )
print ( '\n type( df_counts_sparse ) ', type ( df_counts_sparse ))
print ( '\n df_counts_sparse.sparse.density: ', df_counts_sparse.sparse.density)

 df_counts_sparse:
                 AAACATTGACGACT-1  AAACATTGACGGGA-1  ...  TTTGCATGGATAAG-6  \
ENSEMBL_ID                                           ...                     
ENSG00000239945                 0                 0  ...                 0   
ENSG00000237683                 0                 0  ...                 0   
ENSG00000228463                 0                 0  ...                 0   
ENSG00000235373                 0                 0  ...                 0   
ENSG00000228327                 0                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000212884                 0                 0  ...                 0   
ENSG00000215615                 0                 0  ...                 0   
ENSG00000215764                 0                 0  ...                 0   
ENSG00000215700                 1                 0  ...                 1   
ENSG00000215699                 0            

In [7]:
df_counts_sparse_all_samples_SD_positive = df_counts_sparse[ df_selected_cells.index ].loc [ gene_select_array ]
print ( ' df_counts_sparse_all_samples_SD_positive:' )
print ( df_counts_sparse_all_samples_SD_positive )
print ( '\n type( df_counts_sparse_all_samples_SD_positive ) ', type ( df_counts_sparse_all_samples_SD_positive ))
print ( '\n df_counts_sparse_all_samples_SD_positive.sparse.density: ', df_counts_sparse_all_samples_SD_positive.sparse.density)

del df_counts_sparse

 df_counts_sparse_all_samples_SD_positive:
                 AAACATTGACGACT-1  AAACATTGACGGGA-1  ...  TTTGCATGGATAAG-6  \
ENSEMBL_ID                                           ...                     
ENSG00000254709                 0                 0  ...                 0   
ENSG00000163736                 0                 0  ...                 0   
ENSG00000087086                 3                 6  ...                 8   
ENSG00000163220                 0                 0  ...                 0   
ENSG00000090382                 1                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000060982                 0                 0  ...                 0   
ENSG00000156113                 0                 0  ...                 0   
ENSG00000135549                 0                 0  ...                 0   
ENSG00000155962                 0                 0  ...                 0   
ENSG00000122025      

In [9]:
df_clipped_half_cell_samples_list = []

for sample in  range(2):
  print ( '\n sample: ', sample )
  df_cell_sample = df_selected_cells[[sample]]
  df_cell_select = df_cell_sample [ df_cell_sample[sample] ]
  barcode_list = df_cell_select.index.values.tolist()
  
  df_counts_sparse_sample = df_counts_sparse_all_samples_SD_positive[ barcode_list ]     
  scipy_csr_mat = df_counts_sparse_sample.transpose().sparse.to_coo().tocsr(). astype ( 'float64'  )

  residuals = _pearson_residuals ( scipy_csr_mat, 100, None )
  residual_variance_clipped = np.var( residuals, axis=0)

  df_clipped_half_cell_sample = pd.DataFrame ( index = df_counts_sparse_all_samples_SD_positive.index, data = { sample: residual_variance_clipped  } ) 
  df_clipped_half_cell_sample.sort_values ( [sample], ascending=False, inplace=True )  

  df_clipped_half_cell_samples_list.append ( df_clipped_half_cell_sample )
  
  
df_clipped_half_cell_samples = pd.concat ( df_clipped_half_cell_samples_list, axis=1 )
print ( '\n df_clipped_half_cell_samples' )
print ( df_clipped_half_cell_samples  )
print ( '\n\n df_clipped_half_cell_samples.describe' )
print ( df_clipped_half_cell_samples.describe ( percentiles=pctl_list ) )


 sample:  0

 sample:  1

 df_clipped_half_cell_samples
                         0          1
ENSEMBL_ID                           
ENSG00000163736  46.790085  46.077720
ENSG00000163220  34.098254  33.776977
ENSG00000087086  32.902598  32.463555
ENSG00000090382  30.894894  30.862072
ENSG00000115523  27.900325  28.609090
...                    ...        ...
ENSG00000198682   0.696335   0.762956
ENSG00000233038   0.681522   0.887475
ENSG00000135549   0.671155   1.625988
ENSG00000155962   0.655804   0.628858
ENSG00000122025   0.654775   0.649210

[10908 rows x 2 columns]


 df_clipped_half_cell_samples.describe
                  0             1
count  10908.000000  10908.000000
mean       1.370796      1.375944
std        1.094625      1.099428
min        0.654775      0.628858
1%         0.871125      0.875891
5%         0.958705      0.957452
10%        0.993412      0.994850
25%        1.052163      1.054751
50%        1.161485      1.169532
75%        1.451377      1.450831
90%     

In [10]:
df_clipped_half_cell_samples.to_pickle ( results_dsn )